In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [15]:
data = pd.read_csv('/home/max/15_Рекомендательные системы/lesson_2/webinar_2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [16]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [13]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_probabilities['item_id'])
    probability = np.array(items_probabilities['probability'])
    
    recs = np.random.choice(items, size=n, replace=False, p=probability)
    
    return recs.tolist()

In [37]:
%%time

# your_code
data = data_train
items_probabilities = data.groupby('item_id')['sales_value'].sum().reset_index()
items_probabilities['log'] = np.log1p(items_probabilities['sales_value'])
sum_ = items_probabilities.log.sum()
items_probabilities['probability'] = items_probabilities['log']/sum_.sum()
items_probabilities.probability.sum()

CPU times: user 306 ms, sys: 2.88 ms, total: 308 ms
Wall time: 317 ms


1.0

In [43]:
weighted_random_recommendation(items_probabilities,5)

[6034277, 929421, 13073125, 1100322, 985291]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [100]:
result = pd.read_csv('/home/max/15_Рекомендательные системы/lesson_2/webinar_2/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1000965, 10121560, 9878622, 13512901, 13039531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [101]:
def t(df):
    for j in range(df.shape[1]):
        print(type(df.iloc[0][j]))

In [102]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [103]:
def List(result):
    for i in range(result.shape[0]):
        tt = 'actual'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'random_recommendation'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'popular_recommendation'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'itemitem'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'cosine'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'tfidf'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'own_purchases'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]

In [104]:
t(result)

<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [105]:
List(result)

ValueError: invalid literal for int() with base 10: '  821867   834484   856942   865456   889248   907957   914190   943316\n   951954   954486   958046   962568   969231   971585   979707   986947\n   990656   995242  1004906  1005186  1042083  10503

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [129]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [130]:
# your_code
def random_recommendation(items, n):

    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()
def popularity_recommendation(data, n):

    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [131]:
data = pd.read_csv('/home/max/15_Рекомендательные системы/lesson_2/webinar_2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [132]:
popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity.sort_values('sales_value', ascending=False, inplace=True)